In [1]:
import time
import os
import numpy as np
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # disable infos
os.environ["TF_ENABLE_ONEDNN_OPTS"]="1" # make sure onednn is used on CPU
# tf.config.set_visible_devices([], 'GPU') # GPU improves c-times much

T=3.0
N=int(T*2*12)
Nsim = 10
# N=int(T*4*12) # no real difference
# Nsim = 5
r=0.0303
dtype=np.float32
seed=1
tf.random.set_seed(seed)

t=np.linspace(0,T,N*Nsim,endpoint=True,dtype=dtype).reshape((-1,1))
t=tf.constant(t)

from Commodities import Schwartz2Factor
'Salmon'
# mu, sigma1, sigma2, kappa, alpha, lambda, rho, delta0, P0
# salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.01, 0.9, 0.57, 95] # down,down
salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.2, 0.9, 0.57, 95] # down,up
# salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.6, 0.9, 0.57, 95] # up,up

'Soy'
# mu, sigma1, sigma2, kappa, alpha, lambda, rho, delta0, P0
# soyParam=[0.15, 0.5, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # low vol
soyParam=[0.15, 1, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # medium vol
# soyParam=[0.15, 2, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # high vol

'Risk neutral dynamics'
salmonParam[0]=r
soyParam[0]=r

"Fish feeding 25% of production cost, disease 30%, harvest 10%. Total production cost = 50% of price = labor, smolt, ..."
salmonPrice=salmonParam[-1] #NOK/KG
harvestingCosts=salmonPrice*0.5*0.1 # roughly 10%
feedingCosts=salmonPrice*0.5*0.25
initialSalmon=0.5*salmonPrice+feedingCosts+harvestingCosts #we add the costs to salmon price since they are respected in the model, other costs are fixed and thus removed
salmonParam[-1]=initialSalmon
print(f'Feeding costs {feedingCosts} and Harvesting costs {harvestingCosts}')


soy=Schwartz2Factor(soyParam,t,dtype=dtype)
salmon=Schwartz2Factor(salmonParam,t,dtype=dtype)


from Harvest import Harvest
hc = harvestingCosts
harvest = Harvest(hc)

from Growth import Bertalanffy
wInf=6
a=1.113
b=1.097
c=1.43
growth = Bertalanffy(t,wInf,a,b,c)

from Price import Price
price = Price(salmon)

from Feed import StochFeed,DetermFeed
cr=1.1
fc=feedingCosts
feed_s = StochFeed(fc,cr,r,t,soy)
feed_d = DetermFeed(fc,cr,r,t,soy)

from Mortality import ConstMortatlity,HostParasite,DetermHostParasite
# n0=10000
# m=0.1
# mort = ConstMortatlity(t,n0,m) # not comparable to the other models, because of neglected treatment costs

params=[0.05,0.1,8.71,0.05]
beta=[0.0835,0.0244]
H0=10000.0
P0=1
mort_s = HostParasite(t,params,beta,H0,P0)
mort_d = DetermHostParasite(t,params,beta,H0,P0)

from FishFarm import fishFarm
farm_ss = fishFarm(growth,feed_s,price,harvest,mort_s,stride=Nsim,seed=seed)
farm_sd = fishFarm(growth,feed_s,price,harvest,mort_d,stride=Nsim,seed=seed)
farm_ds = fishFarm(growth,feed_d,price,harvest,mort_s,stride=Nsim,seed=seed)
farm_dd = fishFarm(growth,feed_d,price,harvest,mort_d,stride=Nsim,seed=seed)

from OptimalStopping import Polynomial,DeepOptS,LSMC
batch_size=2**12 #need to fix it for simplicity
batches=20 # not so relevant since we make pathwise comparison, only relevant for value of opt stopping

Feeding costs 11.875 and Harvesting costs 4.75


c:\ProgramData\Anaconda3\envs\intelpython310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
farm_ss.seed(seed)
tf.random.set_seed(seed)
X_ss,V_ss,Vh_ss,ft_ss = farm_ss.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_ss=np.array(X_ss)
V_ss=np.array(V_ss)
Vh_ss=np.array(Vh_ss)
ft_ss=np.array(ft_ss)
print(X_ss.shape)

(145, 81920, 6)


In [3]:
farm_sd.seed(seed)
tf.random.set_seed(seed)
X_sd,V_sd,Vh_sd,ft_sd = farm_sd.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_sd=np.array(X_sd)
V_sd=np.array(V_sd)
Vh_sd=np.array(Vh_sd)
ft_sd=np.array(ft_sd)
print(X_sd.shape)

(145, 81920, 4)


In [4]:
farm_ds.seed(seed)
tf.random.set_seed(seed)
X_ds,V_ds,Vh_ds,ft_ds = farm_ds.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_ds=np.array(X_ds)
V_ds=np.array(V_ds)
Vh_ds=np.array(Vh_ds)
ft_ds=np.array(ft_ds)
print(X_ds.shape)

(145, 81920, 4)


In [5]:
farm_dd.seed(seed)
tf.random.set_seed(seed)
X_dd,V_dd,Vh_dd,ft_dd = farm_dd.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_dd=np.array(X_dd)
V_dd=np.array(V_dd)
Vh_dd=np.array(Vh_dd)
ft_dd=np.array(ft_dd)
print(X_dd.shape)

(145, 81920, 2)


In [6]:
batches=20
farm_ss.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_ss=DeepOptS(r,farm_ss.tCoarse,farm_ss.generateFishFarm,d=farm_ss.d,batch_size=batch_size)
optDeep_ss.train(batches)

tic=time.time()
tauDOS_ss,VtauDOS_ss=optDeep_ss.evaluate(X_ss,V_ss,Vh_ss,ft_ss)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_ss)} with mean value {np.mean(VtauDOS_ss)} in {ctimeEval} s')


100%|██████████| 1506/1506 [01:44<00:00, 14.40epoch/s, loss=-1.99e+6, payoff=2e+6]   


Mean stopping time 1.9975874423980713 with mean value 1982042.0 in 1.6067566871643066 s


In [7]:
batches=20
farm_sd.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_sd=DeepOptS(r,farm_sd.tCoarse,farm_sd.generateFishFarm,d=farm_sd.d,batch_size=batch_size)
optDeep_sd.train(batches)

tic=time.time()
tauDOS_sd,VtauDOS_sd=optDeep_sd.evaluate(X_sd,V_sd,Vh_sd,ft_sd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_sd)} with mean value {np.mean(VtauDOS_sd)} in {ctimeEval} s')


100%|██████████| 1504/1504 [01:00<00:00, 24.84epoch/s, loss=-1.95e+6, payoff=1.97e+6]


Mean stopping time 1.9981629848480225 with mean value 1961629.25 in 1.3286232948303223 s


In [8]:
batches=20
farm_ds.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_ds=DeepOptS(r,farm_ds.tCoarse,farm_ds.generateFishFarm,d=farm_ds.d,batch_size=batch_size)
optDeep_ds.train(batches)

tic=time.time()
tauDOS_ds,VtauDOS_ds=optDeep_ds.evaluate(X_ds,V_ds,Vh_ds,ft_ds)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_ds)} with mean value {np.mean(VtauDOS_ds)} in {ctimeEval} s')


100%|██████████| 1504/1504 [00:57<00:00, 26.18epoch/s, loss=-1.88e+6, payoff=1.92e+6]


Mean stopping time 2.0074455738067627 with mean value 1906331.625 in 1.3013439178466797 s


In [9]:
batches=20
farm_dd.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_dd=DeepOptS(r,farm_dd.tCoarse,farm_dd.generateFishFarm,d=farm_dd.d,batch_size=batch_size)
optDeep_dd.train(batches)

tic=time.time()
tauDOS_dd,VtauDOS_dd=optDeep_dd.evaluate(X_dd,V_dd,Vh_dd,ft_dd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_dd)} with mean value {np.mean(VtauDOS_dd)} in {ctimeEval} s')


100%|██████████| 1502/1502 [00:55<00:00, 26.88epoch/s, loss=-1.88e+6, payoff=1.9e+6] 


Mean stopping time 2.0131049156188965 with mean value 1886896.625 in 1.0873491764068604 s


In [10]:
tComp,Vcomp = fishFarm.compareStoppingTimes(V_ss,[tauDOS_ss,tauDOS_sd,tauDOS_ds,tauDOS_dd],np.array(farm_ss.tCoarse))
for i in range(0,len(tComp)):
    print(f'Mean tau: {tComp[i]} with mean {Vcomp[i]} with ratio {Vcomp[0]/Vcomp[i]}')

Mean tau: [1.99776535] with mean 1982041.8806961246 with ratio 1.0
Mean tau: [1.99834576] with mean 1964776.088978155 with ratio 1.008787663802927
Mean tau: [2.00766411] with mean 1905468.2930239202 with ratio 1.0401862303101799
Mean tau: [2.01333961] with mean 1889590.4454809553 with ratio 1.0489267054859805


In [11]:
basis_ss = Polynomial(deg=2,dtype=dtype)
basis_sd = Polynomial(deg=2,dtype=dtype)
basis_ds = Polynomial(deg=2,dtype=dtype)
basis_dd = Polynomial(deg=2,dtype=dtype)
optLSMC_ss=LSMC(r,farm_ss.tCoarse,farm_ss.generateFishFarm,basis_ss,batch_size=batch_size)
optLSMC_sd=LSMC(r,farm_sd.tCoarse,farm_sd.generateFishFarm,basis_sd,batch_size=batch_size)
optLSMC_ds=LSMC(r,farm_ds.tCoarse,farm_ds.generateFishFarm,basis_ds,batch_size=batch_size)
optLSMC_dd=LSMC(r,farm_dd.tCoarse,farm_dd.generateFishFarm,basis_dd,batch_size=batch_size)

tic=time.time()
tau_ss,Vtau_ss=optLSMC_ss.evaluate(X_ss,V_ss,Vh_ss,ft_ss)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_ss)} with mean value {np.mean(Vtau_ss)} in {ctimeEval} s')

tic=time.time()
tau_sd,Vtau_sd=optLSMC_sd.evaluate(X_sd,V_sd,Vh_sd,ft_sd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_sd)} with mean value {np.mean(Vtau_sd)} in {ctimeEval} s')

tic=time.time()
tau_ds,Vtau_ds=optLSMC_ds.evaluate(X_ds,V_ds,Vh_ds,ft_ds)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_ds)} with mean value {np.mean(Vtau_ds)} in {ctimeEval} s')

tic=time.time()
tau_dd,Vtau_dd=optLSMC_dd.evaluate(X_dd,V_dd,Vh_dd,ft_dd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_dd)} with mean value {np.mean(Vtau_dd)} in {ctimeEval} s')

Mean stopping time 1.9186025857925415 with mean value 1942447.625 in 13.752725839614868 s
Mean stopping time 2.0241072177886963 with mean value 1963525.0 in 3.7595484256744385 s
Mean stopping time 1.9179928302764893 with mean value 1871000.75 in 3.739718437194824 s
Mean stopping time 2.046623706817627 with mean value 1897935.75 in 3.566362142562866 s


In [12]:
tComp,Vcomp = fishFarm.compareStoppingTimes(V_ss,[tau_ss,tau_sd,tau_ds,tau_dd],np.array(farm_ss.tCoarse))
for i in range(0,len(tComp)):
    print(f'Mean tau: {tComp[i]} with mean {Vcomp[i]} with ratio {Vcomp[0]/Vcomp[i]}')

Mean tau: 1.9186025305267322 with mean 1942447.5779450655 with ratio 1.0
Mean tau: 2.02410718818719 with mean 1966792.2884782075 with ratio 0.9876221242701848
Mean tau: 1.917992857829813 with mean 1870282.9516929626 with ratio 1.0385848709077843
Mean tau: 2.0466235583087835 with mean 1900445.335056305 with ratio 1.0221012633796784
